# Evaluate grid point wise estimated lms

## Steps
- develop some functions/classes to make process efficient
- investigate key quantities of interest
- investigate spatial and temporal characteristics

## Guiding questions:
1. Which areas profitted most from specific changes in observational suite?
    - plot segment and segment interaction terms on grid
    - for each grid point, plot which coefficient is largest
    - show overall change in log-variance estimated (how to do this properly?)
2. Which areas are most affected by seasonal changes?
    - plot season coefficients for each grid point
3. Investigate land-ocean contrast
    - use land-sea mask to differentiate and use scatter plots of relevant
    coefficients

## Some classes and functions

In [1]:
source("../RScripts/config.r")
source("../RScripts/plot_functions.r")

# Load all models
lm_data <- load_lm_objects()



Attache Paket: ‘dplyr’


Die folgenden Objekte sind maskiert von ‘package:stats’:

    filter, lag


Die folgenden Objekte sind maskiert von ‘package:base’:

    intersect, setdiff, setequal, union


Linking to GEOS 3.11.1, GDAL 3.6.2, PROJ 9.1.1; sf_use_s2() is TRUE

Lade nötiges Paket: viridisLite

Warning message in mean.default(f):
“Argument ist weder numerisch noch boolesch: gebe NA zurück”


ERROR: [1m[33mError[39m in `map()` at purrr/R/superseded-map-df.R:68:3:[22m
[1m[22m[36mℹ[39m In index: 1.
[1mCaused by error in `var()`:[22m
[33m![39m 'x' ist NULL


In [10]:
lm_files <- list.files(LM_DIR, pattern = "^lm.*\\.Rds", full.names = TRUE)
coords <- stringr::str_extract_all(lm_files, "-?\\d+\\.?\\d*",
    simplify = TRUE
)
lats <- as.numeric(coords[, 1])
lons <- as.numeric(coords[, 2])

In [14]:
lm1 <- readRDS(lm_files[1])
coef(lm1)

segment1              segment2              segment3 
         3.145118e+02          3.024652e+02          4.151815e+01 
             segment4              segment5                  year 
         2.419223e+01          5.769894e+01         -1.577266e-01 
              sin_doy               cos_doy         segment2:year 
         4.078842e+01          1.350623e+02          6.170760e-03 
        segment3:year         segment4:year         segment5:year 
         1.396491e-01          1.482326e-01          1.313623e-01 
     segment2:sin_doy      segment3:sin_doy      segment4:sin_doy 
        -8.921452e+00         -4.097019e+01         -2.903170e+01 
     segment5:sin_doy      segment2:cos_doy      segment3:cos_doy 
        -3.235904e+01         -4.016432e+01         -1.406627e+02 
     segment4:cos_doy      segment5:cos_doy          year:sin_doy 
        -1.284537e+02         -1.232811e+02         -2.089509e-02 
         year:cos_doy segment2:year:sin_doy segment3:year:sin_doy 
        -6.931761e-02          4.576596e-03          2.099216e-02 
segment4:year:sin_doy segment5:year:sin_doy segment2:year:cos_doy 
         1.499229e-02          1.668421e-02          2.070365e-02 
segment3:year:cos_doy segment4:year:cos_doy segment5:year:cos_doy 
         7.209603e-02          6.590923e-02          6.328117e-02

In [11]:
extract_lm_params <- function(file) {
    lm_obj <- readRDS(file)
    
    # Extract summary statistics
    lm_summary <- summary(lm_obj)
    
    tibble(
        coefficients = list(coef(lm_obj)),  # Store as list-column for flexibility
        df_residual = lm_summary$df[2],
        r_squared = lm_summary$r.squared,
        adj_r_squared = lm_summary$adj.r.squared,
        sigma = lm_summary$sigma,
        fstatistic = list(lm_summary$fstatistic)  # Store as list to preserve structure
    )
}

# Process all models and combine with coordinates
lm_data <- map_dfr(lm_files, extract_lm_params) %>%
    mutate(lat = lats, lon = lons)

# Print or inspect the final dataframe
print(lm_data)

Warning message in mean.default(f):
“Argument ist weder numerisch noch boolesch: gebe NA zurück”


ERROR: [1m[33mError[39m in `map()` at purrr/R/superseded-map-df.R:68:3:[22m
[1m[22m[36mℹ[39m In index: 1.
[1mCaused by error in `var()`:[22m
[33m![39m 'x' ist NULL


In [ ]:
# Extract segment coefficients (adjust regex for your model terms)
segment_coefs <- lm_data %>%
    mutate(
        segment_effect = purrr::map_dbl(coefficients, ~ .x[grepl("segment", names(.x))][1]),
        segment_p = purrr::map_dbl(p_values, ~ .x[grepl("segment", names(.x))][1])
    ) %>%
    mutate(significant = ifelse(segment_p < 0.05, "Yes", "No"))

# Generate plot
p <- plot_spatial(segment_coefs, "segment_effect", "Impact of Observation System Jumps") +
    geom_point(
        data = filter(segment_coefs, significant == "Yes"),
        aes(x = lon, y = lat), size = 0.1, color = "white"
    )

save_publication_plot(p, "segment_effects.pdf")
